# Tarefa 2b: resumo de textos abstrativo

Neste caderno, você gerencia os desafios decorrentes do resumo de documentos grandes. O texto de entrada pode exceder o tamanho do contexto do modelo, gerar saídas alucinadas ou acionar erros de falta de memória.

Para mitigar esses problemas, esse caderno demonstra uma arquitetura que usa fragmentação e encadeamento de prompts com o framework [LangChain] (https://python.langchain.com/docs/get_started/introduction.html), um toolkit que permite que aplicações usem modelos de linguagem.

Você verá uma abordagem de cenários em que os documentos do usuário ultrapassam os limites de tokens. A fragmentação divide os documentos em segmentos menores do que os limites de comprimento do contexto antes de alimentá-los sequencialmente nos modelos. Isso encadeia prompts em blocos, mantendo o contexto anterior. Você aplica essa abordagem para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, publicações de blogs e outros conteúdos relevantes.

## Tarefa 2b.1: configurar o ambiente

Nesta tarefa, você configurará o ambiente.

In [1]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tarefa 2b.2: resumo de texto longo 

### Configurar o LangChain com o Boto3

Nesta tarefa, você precisará especificar o LLM para a classe do Bedrock do LangChain e pode transmitir argumentos para inferência.

In [75]:
# model configuration
from langchain_aws import ChatBedrock
modelId = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"

llm = ChatBedrock(
    model_id=modelId,
    model_kwargs={
        "max_tokens": 2048,
        "temperature": 0,
        "top_p": 1
    },
    client=bedrock_client
)


## Tarefa 2b.3: carregar um arquivo de texto com muitos tokens

Nesta tarefa, você poderá encontrar um arquivo de texto da [carta do CEO da Amazon aos acionistas em 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) no diretório de cartas. A célula a seguir carrega o arquivo de texto e conta o número de tokens. Você verá um aviso indicando que o número de tokens no arquivo de texto excede o número máximo de tokens para esse modelo.

In [76]:
#get tokens
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r", encoding="utf-8") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

e:\workspace-dev-ia\noteboocks-AWS-arquiteturas\.venv\Lib\site-packages\langchain_aws\chat_models\bedrock.py:1092: UserWarning: Falling back to default token method due to missing or incompatible `anthropic` installation (needs <=0.38.0).

If using `anthropic>0.38.0`, it is recommended to provide the model class with a custom_get_token_ids method implementing a more accurate tokenizer for Anthropic. For get_num_tokens, as another alternative, you can implement your own token counter method using the ChatAnthropic or AnthropicLLM classes.
  warnings.warn(


6526

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** você pode ignorar os avisos com segurança e prosseguir para a próxima célula.

## Tarefa 2b.4: dividir o texto longo em blocos

Nesta tarefa, você dividirá o texto em partes menores porque ele é muito longo para caber no prompt. `RecursiveCharacterTextSplitter` no LangChain oferece suporte à divisão de texto longo em partes recursivamente até que o tamanho de cada bloco se torne menor que `chunk_size`. Um texto é separado com `separators=["\n\n", "\n"]` em partes, o que evita dividir cada parágrafo em várias partes.

Ao usar 6 mil caracteres por bloco, você pode resumir cada parte separadamente. O número de tokens, ou pedaços de palavras, em um bloco dependerá do texto.

In [77]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [78]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

Now we have 10 documents and the first one has 439 tokens


## Tarefa 2b.5: resumir e reunir blocos

Nesta tarefa, supondo que o número de tokens seja consistente nos outros documentos, você deverá ter tudo pronto para começar. Você pode usar o `load_summarize_chain` do LangChain para resumir o texto. `load_summarize_chain` fornece três formas de resumo: `stuff`, `map_reduce` e `refine`.

- `stuff`: coloca todos os blocos em um único prompt. Sendo assim, isso atingiria o limite máximo de tokens.
- `map_reduce`: resume cada bloco, combina os resumos e resume o resumo combinado. Quando o resumo combinado é muito grande, isso gera um erro.
- `refine`: resume o primeiro bloco e depois resume o segundo bloco com o primeiro resumo. O mesmo processo se repete até que todos os blocos sejam resumidos.

Tanto map_reduce quanto refinamento invocam o LLM várias vezes e levam tempo para obter o resumo final. Você pode tentar map_reduce aqui.

In [79]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Observação:** dependendo do número de documentos, da cota de taxa de solicitações do Bedrock e das configurações de novas tentativas definidas, o processamento da cadeia abaixo pode levar algum tempo.

In [80]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory, and with some of our own operating challenges to boot, we still found a way to grow demand (on top of the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest businesses to meaningfully improve customer experience short and long term. And, we made important adjustments in our investment decisions and the way in which we’ll invent moving forward, while still preserving the long-term investments that we believe can change the future of Amazon for customers, shareholders, and employees.

While there were an unusual number of simultaneous challenges this past year, the reality is tha

e:\workspace-dev-ia\noteboocks-AWS-arquiteturas\.venv\Lib\site-packages\langchain_aws\chat_models\bedrock.py:1092: UserWarning: Falling back to default token method due to missing or incompatible `anthropic` installation (needs <=0.38.0).

If using `anthropic>0.38.0`, it is recommended to provide the model class with a custom_get_token_ids method implementing a more accurate tokenizer for Anthropic. For get_num_tokens, as another alternative, you can implement your own token counter method using the ChatAnthropic or AnthropicLLM classes.
  warnings.warn(



> Finished chain.

> Finished chain.


In [81]:
# print output
print(output['output_text'])

Here's a concise summary:

Amazon's 2022 shareholder letter outlines the company's resilience and strategic evolution amid economic challenges. Key points include:

1. Business Adaptation:
- Conducted comprehensive review leading to closure of unprofitable ventures and 27,000 job cuts
- Implemented return-to-office policy and optimized fulfillment network
- Restructured into eight regional networks for improved efficiency

2. Core Growth Areas:
- AWS continues strong growth ($85B revenue) despite temporary slowdown
- Advertising business showing robust performance ($31B revenue, 25% YoY growth)
- Expanding into grocery and B2B procurement markets
- Launched Buy with Prime and entering healthcare sector

3. Future Initiatives:
- Project Kuiper for satellite internet
- Heavy investment in AI and Large Language Models
- Focus on international expansion

The overall message emphasizes Amazon's strategy of balancing cost optimization with long-term investment in promising technologies and m

Você testou o uso da fragmentação e do encadeamento de prompts com o framework LangChain para resumir documentos grandes e, ao mesmo tempo, mitigar problemas decorrentes de textos longos de entrada.

### Experimente você mesmo
- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Teste o comprimento do token para entender a latência e a responsividade do serviço.
- Aplique diferentes princípios de engenharia de prompts para gerar resultados melhores.

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 3**.